In [2]:
import os
os.getcwd()
os.chdir("C:\Data")
os.getcwd()

'C:\\Data'

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from patsy import dmatrix
from pyearth import Earth
from sklearn.metrics import (mean_squared_error, r2_score, roc_curve, auc, precision_recall_curve, make_scorer,
                             recall_score, accuracy_score, precision_score, confusion_matrix)
from sklearn.model_selection import (cross_val_score, train_test_split, KFold, StratifiedKFold,
                                     GridSearchCV, ParameterGrid)
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.ensemble import (BaggingRegressor, BaggingClassifier, RandomForestRegressor, RandomForestClassifier, 
                              GradientBoostingRegressor, GradientBoostingClassifier, AdaBoostRegressor, AdaBoostClassifier, 
                              VotingRegressor, VotingClassifier, StackingRegressor, StackingClassifier)
from sklearn.linear_model import LinearRegression, LogisticRegression, LassoCV, RidgeCV, ElasticNetCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
import itertools as it
import xgboost as xgb
import time as time
import random
from skimpy import clean_columns
from statsmodels.stats.outliers_influence import variance_inflation_factor

#Libraries for visualizing trees
from sklearn.tree import export_graphviz 
from six import StringIO
from IPython.display import Image  
import pydotplus

from sklearn.preprocessing import StandardScaler
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission.head()

,id,y
0,5380,8.608222
1,5381,8.608222
2,5382,8.608222
3,5383,8.608222
4,5384,8.608222


In [5]:
train.corr()['y'].sort_values(ascending = False)

y       1.000000
x146    0.378696
x102    0.378436
x014    0.364737
x581    0.346549
          ...   
x465         NaN
x518         NaN
x594         NaN
x643         NaN
x703         NaN
Name: y, Length: 756, dtype: float64

In [6]:
train.drop(columns = ['x063', 'x137', 'x255', 'x385', 'x405', 'x453', 
                      'x465', 'x518', 'x594', 'x643']).dropna()
test.drop(columns = ['x063', 'x137', 'x255', 'x385', 'x405', 'x453', 'x465', 
                    'x518', 'x594', 'x643'])
train.dropna()
test.dropna()

,id,x001,x002,x003,x004,x005,x006,x007,x008,x009,...,x756,x757,x758,x759,x760,x761,x762,x763,x764,x765
2,5382,3.888719e+10,7495.57,7.15,0.00285,2.160400e+09,0.42,351570.67,841523.0,170240.0,...,0.0100,0.0017,10847937619,0.83,35.0,1,19.0,7.8513,21.27,19.09
11,5391,1.584968e+10,54672.80,5.99,0.00180,1.584968e+09,0.49,21247099.66,7097366.0,4082.0,...,0.0000,0.0005,47600000000000,0.01,9.0,0,6.0,7.7204,372.37,-0.19
13,5393,1.190000e+11,6538.40,6.39,0.00490,5.414583e+09,0.34,563532.49,669825.0,518.0,...,0.0145,0.0009,3385667,0.32,10.0,4,3.0,15.2255,31.51,-0.55
16,5396,2.850000e+11,33256.08,7.14,0.00780,2.686603e+09,0.25,3258294.21,4672528.0,192249.0,...,0.2477,0.0038,904000000000,0.27,49.0,12,28.0,83.8195,127.99,0.20
18,5398,4.714020e+10,4044.12,6.48,0.00140,2.142736e+09,0.21,771895.45,565300.0,16161.0,...,0.1605,0.0003,151029283,0.15,5.0,3,4.0,17.5843,24.01,-0.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4393,9773,1.773297e+10,11274.47,5.09,0.00025,2.216621e+09,0.33,5477016.53,1129652.0,15944.0,...,0.0200,0.0004,298000000000,0.11,7.0,1,6.0,5.8391,157.45,-0.11
4395,9775,1.380000e+11,2978.24,6.87,0.01600,1.896666e+09,0.37,284408.83,298714.0,120817.0,...,0.1713,0.0014,1187967953,0.76,37.0,8,18.0,57.3197,11.71,-0.38
4396,9776,2.743338e+10,27593.97,6.02,0.00245,5.486677e+09,0.40,10605094.08,4470217.0,7388734.0,...,1.1565,0.6084,2160000000000,0.57,1860.0,1,1042.0,2.9667,270.52,-0.75
4400,9780,8.556628e+09,7518.26,6.74,0.00005,2.852209e+09,0.00,559939.70,911940.0,105542.0,...,0.2890,0.0010,551000000000,0.67,25.0,1,9.0,2.5226,29.27,0.32


In [7]:
train_cleaned = train.drop(columns = ['id', 'x063', 'x137', 'x255', 'x385', 'x405', 'x453', 
                      'x465', 'x518', 'x594', 'x643', 'x703']).dropna()
train_cleaned

,x001,x002,x003,x004,x005,x006,x007,x008,x009,x010,...,x757,x758,x759,x760,x761,x762,x763,x764,x765,y
0,9.681860e+10,6991.15,7.76,0.00380,5.378811e+09,0.31,266117.20,934577.0,14539.0,26900000000000,...,0.0007,297281012,0.13,5.0,5,2.0,8.5127,14.28,-0.75,5
2,3.218944e+10,3991.98,5.77,0.00010,2.476111e+09,0.00,774385.01,375738.0,144143.0,135000000000000,...,0.0029,100474819,0.39,39.0,2,18.0,9.6800,25.06,-0.49,11
4,3.063412e+10,3621.00,7.52,0.00060,1.392460e+09,0.21,169860.29,474253.0,17914.0,6000000000000,...,0.0005,109546590,0.11,11.0,1,3.0,16.2717,5.81,-0.42,5
5,4.377769e+10,27776.26,6.02,0.00505,5.472212e+09,0.50,10797026.17,4501083.0,7538720.0,229000000000000000,...,0.6223,154000000000000,0.52,1883.0,1,1055.0,4.5550,271.84,0.62,1
7,1.583740e+10,17060.72,5.73,0.00275,2.262486e+09,0.49,10151935.50,2262318.0,4887848.0,131000000000000000,...,0.4630,7170000000000,0.86,2007.0,1,1113.0,5.5842,137.96,-0.94,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5365,4.002112e+10,6976.24,7.54,0.00110,2.001056e+09,0.36,613298.58,997603.0,641036.0,170000000000000,...,0.0177,2017905,0.02,115.0,4,63.0,15.6049,17.96,-0.40,2
5367,7.565784e+09,36011.48,5.52,0.00070,1.513157e+09,0.40,20865114.16,4762874.0,395329.0,1410000000000000000,...,0.0409,1950000000000,0.02,228.0,2,136.0,3.6861,389.56,15.03,13
5370,4.196064e+10,17683.59,6.88,0.00080,1.998125e+09,0.29,3142310.94,2685423.0,42946.0,8010000000000000,...,0.0009,5893164787,0.51,15.0,2,9.0,15.4456,93.46,0.11,4
5376,9.279017e+10,21572.94,6.96,0.00120,3.093006e+09,0.30,2649164.57,2934417.0,19106.0,7220000000000000,...,0.0003,36089167,0.01,6.0,4,4.0,23.6890,76.05,-0.90,8


In [8]:
test_cleaned = test.drop(columns = ['id','x063', 'x137', 'x255', 'x385', 'x405', 'x453', 
                        'x465', 'x518', 'x594', 'x643', 'x703']).dropna()
test_cleaned

,x001,x002,x003,x004,x005,x006,x007,x008,x009,x010,...,x756,x757,x758,x759,x760,x761,x762,x763,x764,x765
2,3.888719e+10,7495.57,7.15,0.00285,2.160400e+09,0.42,351570.67,841523.0,170240.0,41400000000000,...,0.0100,0.0017,10847937619,0.83,35.0,1,19.0,7.8513,21.27,19.09
11,1.584968e+10,54672.80,5.99,0.00180,1.584968e+09,0.49,21247099.66,7097366.0,4082.0,1290000000000000000,...,0.0000,0.0005,47600000000000,0.01,9.0,0,6.0,7.7204,372.37,-0.19
13,1.190000e+11,6538.40,6.39,0.00490,5.414583e+09,0.34,563532.49,669825.0,518.0,108000000000000,...,0.0145,0.0009,3385667,0.32,10.0,4,3.0,15.2255,31.51,-0.55
16,2.850000e+11,33256.08,7.14,0.00780,2.686603e+09,0.25,3258294.21,4672528.0,192249.0,20800000000000000,...,0.2477,0.0038,904000000000,0.27,49.0,12,28.0,83.8195,127.99,0.20
18,4.714020e+10,4044.12,6.48,0.00140,2.142736e+09,0.21,771895.45,565300.0,16161.0,152000000000000,...,0.1605,0.0003,151029283,0.15,5.0,3,4.0,17.5843,24.01,-0.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4393,1.773297e+10,11274.47,5.09,0.00025,2.216621e+09,0.33,5477016.53,1129652.0,15944.0,23600000000000000,...,0.0200,0.0004,298000000000,0.11,7.0,1,6.0,5.8391,157.45,-0.11
4395,1.380000e+11,2978.24,6.87,0.01600,1.896666e+09,0.37,284408.83,298714.0,120817.0,12600000000000,...,0.1713,0.0014,1187967953,0.76,37.0,8,18.0,57.3197,11.71,-0.38
4396,2.743338e+10,27593.97,6.02,0.00245,5.486677e+09,0.40,10605094.08,4470217.0,7388734.0,225000000000000000,...,1.1565,0.6084,2160000000000,0.57,1860.0,1,1042.0,2.9667,270.52,-0.75
4400,8.556628e+09,7518.26,6.74,0.00005,2.852209e+09,0.00,559939.70,911940.0,105542.0,122000000000000,...,0.2890,0.0010,551000000000,0.67,25.0,1,9.0,2.5226,29.27,0.32


In [10]:
train_cleaned

,x001,x002,x003,x004,x005,x006,x007,x008,x009,x010,...,x757,x758,x759,x760,x761,x762,x763,x764,x765,y
0,9.681860e+10,6991.15,7.76,0.00380,5.378811e+09,0.31,266117.20,934577.0,14539.0,26900000000000,...,0.0007,297281012,0.13,5.0,5,2.0,8.5127,14.28,-0.75,5
2,3.218944e+10,3991.98,5.77,0.00010,2.476111e+09,0.00,774385.01,375738.0,144143.0,135000000000000,...,0.0029,100474819,0.39,39.0,2,18.0,9.6800,25.06,-0.49,11
4,3.063412e+10,3621.00,7.52,0.00060,1.392460e+09,0.21,169860.29,474253.0,17914.0,6000000000000,...,0.0005,109546590,0.11,11.0,1,3.0,16.2717,5.81,-0.42,5
5,4.377769e+10,27776.26,6.02,0.00505,5.472212e+09,0.50,10797026.17,4501083.0,7538720.0,229000000000000000,...,0.6223,154000000000000,0.52,1883.0,1,1055.0,4.5550,271.84,0.62,1
7,1.583740e+10,17060.72,5.73,0.00275,2.262486e+09,0.49,10151935.50,2262318.0,4887848.0,131000000000000000,...,0.4630,7170000000000,0.86,2007.0,1,1113.0,5.5842,137.96,-0.94,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5365,4.002112e+10,6976.24,7.54,0.00110,2.001056e+09,0.36,613298.58,997603.0,641036.0,170000000000000,...,0.0177,2017905,0.02,115.0,4,63.0,15.6049,17.96,-0.40,2
5367,7.565784e+09,36011.48,5.52,0.00070,1.513157e+09,0.40,20865114.16,4762874.0,395329.0,1410000000000000000,...,0.0409,1950000000000,0.02,228.0,2,136.0,3.6861,389.56,15.03,13
5370,4.196064e+10,17683.59,6.88,0.00080,1.998125e+09,0.29,3142310.94,2685423.0,42946.0,8010000000000000,...,0.0009,5893164787,0.51,15.0,2,9.0,15.4456,93.46,0.11,4
5376,9.279017e+10,21572.94,6.96,0.00120,3.093006e+09,0.30,2649164.57,2934417.0,19106.0,7220000000000000,...,0.0003,36089167,0.01,6.0,4,4.0,23.6890,76.05,-0.90,8


In [11]:
train_cleaned.corr()['y'].sort_values(ascending = False)

y       1.000000
x102    0.386480
x146    0.386226
x619    0.365515
x687    0.365401
          ...   
x685   -0.248506
x488   -0.251006
x654   -0.269024
x742   -0.289128
x096   -0.315246
Name: y, Length: 744, dtype: float64

In [15]:
correlation_matrix = train_cleaned.corr()
missing_columns = set(train_cleaned.columns) - set(correlation_matrix.columns)
train_full = train_cleaned.drop(columns=missing_columns)

In [16]:
train_full

,x001,x002,x003,x004,x005,x006,x007,x008,x009,x010,...,x757,x758,x759,x760,x761,x762,x763,x764,x765,y
0,9.681860e+10,6991.15,7.76,0.00380,5.378811e+09,0.31,266117.20,934577.0,14539.0,26900000000000,...,0.0007,297281012,0.13,5.0,5,2.0,8.5127,14.28,-0.75,5
2,3.218944e+10,3991.98,5.77,0.00010,2.476111e+09,0.00,774385.01,375738.0,144143.0,135000000000000,...,0.0029,100474819,0.39,39.0,2,18.0,9.6800,25.06,-0.49,11
4,3.063412e+10,3621.00,7.52,0.00060,1.392460e+09,0.21,169860.29,474253.0,17914.0,6000000000000,...,0.0005,109546590,0.11,11.0,1,3.0,16.2717,5.81,-0.42,5
5,4.377769e+10,27776.26,6.02,0.00505,5.472212e+09,0.50,10797026.17,4501083.0,7538720.0,229000000000000000,...,0.6223,154000000000000,0.52,1883.0,1,1055.0,4.5550,271.84,0.62,1
7,1.583740e+10,17060.72,5.73,0.00275,2.262486e+09,0.49,10151935.50,2262318.0,4887848.0,131000000000000000,...,0.4630,7170000000000,0.86,2007.0,1,1113.0,5.5842,137.96,-0.94,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5365,4.002112e+10,6976.24,7.54,0.00110,2.001056e+09,0.36,613298.58,997603.0,641036.0,170000000000000,...,0.0177,2017905,0.02,115.0,4,63.0,15.6049,17.96,-0.40,2
5367,7.565784e+09,36011.48,5.52,0.00070,1.513157e+09,0.40,20865114.16,4762874.0,395329.0,1410000000000000000,...,0.0409,1950000000000,0.02,228.0,2,136.0,3.6861,389.56,15.03,13
5370,4.196064e+10,17683.59,6.88,0.00080,1.998125e+09,0.29,3142310.94,2685423.0,42946.0,8010000000000000,...,0.0009,5893164787,0.51,15.0,2,9.0,15.4456,93.46,0.11,4
5376,9.279017e+10,21572.94,6.96,0.00120,3.093006e+09,0.30,2649164.57,2934417.0,19106.0,7220000000000000,...,0.0003,36089167,0.01,6.0,4,4.0,23.6890,76.05,-0.90,8


In [17]:
correlation_matrix = train_full.corr()

In [18]:
#dropping variables with high multicollinearity

vif = pd.DataFrame(index=train_full.columns, columns=['VIF'])

for var in train_full.columns:
    mask = train_full.columns != var
    r_squared = 1.0 - correlation_matrix.loc[var, mask].pow(2).sum()
    vif.loc[var, 'VIF'] = 1.0 / (1.0 - r_squared)
    
high_vif_variables = vif[vif['VIF'] > 3]
print(high_vif_variables)

           VIF
x402  3.357982
x658   3.17963


In [19]:
train_full = train_full.drop(columns = ['x402', 'x658'])
train_full

,x001,x002,x003,x004,x005,x006,x007,x008,x009,x010,...,x757,x758,x759,x760,x761,x762,x763,x764,x765,y
0,9.681860e+10,6991.15,7.76,0.00380,5.378811e+09,0.31,266117.20,934577.0,14539.0,26900000000000,...,0.0007,297281012,0.13,5.0,5,2.0,8.5127,14.28,-0.75,5
2,3.218944e+10,3991.98,5.77,0.00010,2.476111e+09,0.00,774385.01,375738.0,144143.0,135000000000000,...,0.0029,100474819,0.39,39.0,2,18.0,9.6800,25.06,-0.49,11
4,3.063412e+10,3621.00,7.52,0.00060,1.392460e+09,0.21,169860.29,474253.0,17914.0,6000000000000,...,0.0005,109546590,0.11,11.0,1,3.0,16.2717,5.81,-0.42,5
5,4.377769e+10,27776.26,6.02,0.00505,5.472212e+09,0.50,10797026.17,4501083.0,7538720.0,229000000000000000,...,0.6223,154000000000000,0.52,1883.0,1,1055.0,4.5550,271.84,0.62,1
7,1.583740e+10,17060.72,5.73,0.00275,2.262486e+09,0.49,10151935.50,2262318.0,4887848.0,131000000000000000,...,0.4630,7170000000000,0.86,2007.0,1,1113.0,5.5842,137.96,-0.94,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5365,4.002112e+10,6976.24,7.54,0.00110,2.001056e+09,0.36,613298.58,997603.0,641036.0,170000000000000,...,0.0177,2017905,0.02,115.0,4,63.0,15.6049,17.96,-0.40,2
5367,7.565784e+09,36011.48,5.52,0.00070,1.513157e+09,0.40,20865114.16,4762874.0,395329.0,1410000000000000000,...,0.0409,1950000000000,0.02,228.0,2,136.0,3.6861,389.56,15.03,13
5370,4.196064e+10,17683.59,6.88,0.00080,1.998125e+09,0.29,3142310.94,2685423.0,42946.0,8010000000000000,...,0.0009,5893164787,0.51,15.0,2,9.0,15.4456,93.46,0.11,4
5376,9.279017e+10,21572.94,6.96,0.00120,3.093006e+09,0.30,2649164.57,2934417.0,19106.0,7220000000000000,...,0.0003,36089167,0.01,6.0,4,4.0,23.6890,76.05,-0.90,8


In [30]:
train['y'].max() - train['y'].min()

99

#applying log in the variables due to the relatively small range of values expressed in the dependent variable.

In [44]:
logtrain = np.log(train_full)
logtrain

,x001,x002,x003,x004,x005,x006,x007,x008,x009,x010,...,x757,x758,x759,x760,x761,x762,x763,x764,x765,y
0,25.296105,8.852400,2.048982,-5.572754,22.405733,-1.171183,12.491692,13.747849,9.584590,30.923147,...,-7.264430,19.510188,-2.040221,1.609438,1.609438,0.693147,2.141559,2.658860,NaN,1.609438
2,24.194904,8.292043,1.752672,-9.210340,21.629955,-inf,13.559824,12.836647,11.878561,32.536296,...,-5.843045,18.425418,-0.941609,3.663562,0.693147,2.890372,2.270062,3.221273,NaN,2.397895
4,24.145380,8.194506,2.017566,-7.418581,21.054338,-1.560648,12.042732,13.069496,9.793338,29.422781,...,-7.600902,18.511860,-2.207275,2.397895,0.000000,1.098612,2.789427,1.759581,NaN,1.609438
5,24.502390,10.231937,1.795087,-5.288367,22.422949,-0.693147,16.194781,15.319829,15.835563,39.972498,...,-0.474333,32.667974,-0.653926,7.540622,0.000000,6.961296,1.516226,5.605214,-0.478036,0.000000
7,23.485640,9.744534,1.745716,-5.896154,21.539730,-0.713350,16.133175,14.631901,15.402263,39.413974,...,-0.770028,29.600927,-0.150823,7.604396,0.000000,7.014814,1.719941,4.926964,NaN,1.098612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5365,24.412673,8.850265,2.020222,-6.812445,21.416941,-1.021651,13.326607,13.813111,13.370841,32.766820,...,-4.034191,14.517570,-3.912023,4.744932,1.386294,4.143135,2.747585,2.888147,NaN,0.693147
5367,22.746902,10.491593,1.708378,-7.264430,21.137464,-0.916291,16.853589,15.376362,12.887474,41.790121,...,-3.196625,28.298850,-3.912023,5.429346,0.693147,4.912655,1.304569,5.965018,2.710048,2.564949
5370,24.459998,9.780392,1.928619,-7.130899,21.415475,-1.237874,14.960469,14.803349,10.667699,36.619467,...,-7.013116,22.497059,-0.673345,2.708050,0.693147,2.197225,2.737324,4.537534,-2.207275,1.386294
5376,25.253606,9.979195,1.940179,-6.725434,21.852409,-1.203973,14.789755,14.892019,9.857758,36.515631,...,-8.111728,17.401503,-4.605170,1.791759,1.386294,1.386294,3.165011,4.331391,NaN,2.079442


In [45]:
logtrain.corr()['y'].sort_values(ascending = True)

x096   -0.367197
x420   -0.362048
x654   -0.351765
x721   -0.331272
x716   -0.313349
          ...   
x570         NaN
x591         NaN
x631         NaN
x651         NaN
x687         NaN
Name: y, Length: 742, dtype: float64

In [48]:
logtrain_1 = logtrain.dropna(axis=1)
correlation_matrix = logtrain_1.corr()
logtrain_1

,x001,x002,x003,x004,x005,x006,x007,x008,x009,x010,...,x756,x757,x758,x759,x760,x761,x762,x763,x764,y
0,25.296105,8.852400,2.048982,-5.572754,22.405733,-1.171183,12.491692,13.747849,9.584590,30.923147,...,0.451521,-7.264430,19.510188,-2.040221,1.609438,1.609438,0.693147,2.141559,2.658860,1.609438
2,24.194904,8.292043,1.752672,-9.210340,21.629955,-inf,13.559824,12.836647,11.878561,32.536296,...,-0.780450,-5.843045,18.425418,-0.941609,3.663562,0.693147,2.890372,2.270062,3.221273,2.397895
4,24.145380,8.194506,2.017566,-7.418581,21.054338,-1.560648,12.042732,13.069496,9.793338,29.422781,...,-4.605170,-7.600902,18.511860,-2.207275,2.397895,0.000000,1.098612,2.789427,1.759581,1.609438
5,24.502390,10.231937,1.795087,-5.288367,22.422949,-0.693147,16.194781,15.319829,15.835563,39.972498,...,-inf,-0.474333,32.667974,-0.653926,7.540622,0.000000,6.961296,1.516226,5.605214,0.000000
7,23.485640,9.744534,1.745716,-5.896154,21.539730,-0.713350,16.133175,14.631901,15.402263,39.413974,...,0.265513,-0.770028,29.600927,-0.150823,7.604396,0.000000,7.014814,1.719941,4.926964,1.098612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5365,24.412673,8.850265,2.020222,-6.812445,21.416941,-1.021651,13.326607,13.813111,13.370841,32.766820,...,-0.631300,-4.034191,14.517570,-3.912023,4.744932,1.386294,4.143135,2.747585,2.888147,0.693147
5367,22.746902,10.491593,1.708378,-7.264430,21.137464,-0.916291,16.853589,15.376362,12.887474,41.790121,...,-2.407946,-3.196625,28.298850,-3.912023,5.429346,0.693147,4.912655,1.304569,5.965018,2.564949
5370,24.459998,9.780392,1.928619,-7.130899,21.415475,-1.237874,14.960469,14.803349,10.667699,36.619467,...,-2.465104,-7.013116,22.497059,-0.673345,2.708050,0.693147,2.197225,2.737324,4.537534,1.386294
5376,25.253606,9.979195,1.940179,-6.725434,21.852409,-1.203973,14.789755,14.892019,9.857758,36.515631,...,-1.500584,-8.111728,17.401503,-4.605170,1.791759,1.386294,1.386294,3.165011,4.331391,2.079442


In [59]:
for var in logtrain_1.columns:
    mask = logtrain_1.columns != var
    r_squared = 1.0 - correlation_matrix.loc[var, mask].pow(2).sum()
    vif.loc[var, 'VIF'] = 1.0 / (1.0 - r_squared)
    
high_vif_variables = vif[vif['VIF'] > 1]
print(high_vif_variables)

           VIF
x075  1.034745
x402  3.357982
x516       inf
x556       inf
x658   3.17963


C:\Users\dylan\AppData\Local\Temp/ipykernel_2492/197459438.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  vif.loc[var, 'VIF'] = 1.0 / (1.0 - r_squared)


In [80]:
df = logtrain_1

# Replace infinity with NaN
df = df.replace([np.inf], np.nan)
df = df.replace([-np.inf], np.nan)

In [81]:
for column in df.columns:
    avg = df[column].mean()
    df[column].fillna(avg, inplace=True)
    
df

,x001,x002,x003,x004,x005,x006,x007,x008,x009,x010,...,x756,x757,x758,x759,x760,x761,x762,x763,x764,y
0,25.296105,8.852400,2.048982,-5.572754,22.405733,-1.171183,12.491692,13.747849,9.584590,30.923147,...,0.451521,-7.264430,19.510188,-2.040221,1.609438,1.609438,0.693147,2.141559,2.658860,1.609438
2,24.194904,8.292043,1.752672,-9.210340,21.629955,-1.059477,13.559824,12.836647,11.878561,32.536296,...,-0.780450,-5.843045,18.425418,-0.941609,3.663562,0.693147,2.890372,2.270062,3.221273,2.397895
4,24.145380,8.194506,2.017566,-7.418581,21.054338,-1.560648,12.042732,13.069496,9.793338,29.422781,...,-4.605170,-7.600902,18.511860,-2.207275,2.397895,0.000000,1.098612,2.789427,1.759581,1.609438
5,24.502390,10.231937,1.795087,-5.288367,22.422949,-0.693147,16.194781,15.319829,15.835563,39.972498,...,-1.681761,-0.474333,32.667974,-0.653926,7.540622,0.000000,6.961296,1.516226,5.605214,0.000000
7,23.485640,9.744534,1.745716,-5.896154,21.539730,-0.713350,16.133175,14.631901,15.402263,39.413974,...,0.265513,-0.770028,29.600927,-0.150823,7.604396,0.000000,7.014814,1.719941,4.926964,1.098612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5365,24.412673,8.850265,2.020222,-6.812445,21.416941,-1.021651,13.326607,13.813111,13.370841,32.766820,...,-0.631300,-4.034191,14.517570,-3.912023,4.744932,1.386294,4.143135,2.747585,2.888147,0.693147
5367,22.746902,10.491593,1.708378,-7.264430,21.137464,-0.916291,16.853589,15.376362,12.887474,41.790121,...,-2.407946,-3.196625,28.298850,-3.912023,5.429346,0.693147,4.912655,1.304569,5.965018,2.564949
5370,24.459998,9.780392,1.928619,-7.130899,21.415475,-1.237874,14.960469,14.803349,10.667699,36.619467,...,-2.465104,-7.013116,22.497059,-0.673345,2.708050,0.693147,2.197225,2.737324,4.537534,1.386294
5376,25.253606,9.979195,1.940179,-6.725434,21.852409,-1.203973,14.789755,14.892019,9.857758,36.515631,...,-1.500584,-8.111728,17.401503,-4.605170,1.791759,1.386294,1.386294,3.165011,4.331391,2.079442


In [89]:
df.corr()['y'].sort_values(ascending = False)

y       1.000000
x118    0.423442
x724    0.423442
x146    0.421050
x670    0.415718
          ...   
x654   -0.345420
x420   -0.348804
x096   -0.367197
x516         NaN
x556         NaN
Name: y, Length: 660, dtype: float64

In [92]:
logtrain_cleaned = df.drop(columns = ['x516','x556'])
correlation_matrix = logtrain_cleaned.corr()['y'].sort_values(ascending = False)
logtrain_cleaned

,x001,x002,x003,x004,x005,x006,x007,x008,x009,x010,...,x756,x757,x758,x759,x760,x761,x762,x763,x764,y
0,25.296105,8.852400,2.048982,-5.572754,22.405733,-1.171183,12.491692,13.747849,9.584590,30.923147,...,0.451521,-7.264430,19.510188,-2.040221,1.609438,1.609438,0.693147,2.141559,2.658860,1.609438
2,24.194904,8.292043,1.752672,-9.210340,21.629955,-1.059477,13.559824,12.836647,11.878561,32.536296,...,-0.780450,-5.843045,18.425418,-0.941609,3.663562,0.693147,2.890372,2.270062,3.221273,2.397895
4,24.145380,8.194506,2.017566,-7.418581,21.054338,-1.560648,12.042732,13.069496,9.793338,29.422781,...,-4.605170,-7.600902,18.511860,-2.207275,2.397895,0.000000,1.098612,2.789427,1.759581,1.609438
5,24.502390,10.231937,1.795087,-5.288367,22.422949,-0.693147,16.194781,15.319829,15.835563,39.972498,...,-1.681761,-0.474333,32.667974,-0.653926,7.540622,0.000000,6.961296,1.516226,5.605214,0.000000
7,23.485640,9.744534,1.745716,-5.896154,21.539730,-0.713350,16.133175,14.631901,15.402263,39.413974,...,0.265513,-0.770028,29.600927,-0.150823,7.604396,0.000000,7.014814,1.719941,4.926964,1.098612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5365,24.412673,8.850265,2.020222,-6.812445,21.416941,-1.021651,13.326607,13.813111,13.370841,32.766820,...,-0.631300,-4.034191,14.517570,-3.912023,4.744932,1.386294,4.143135,2.747585,2.888147,0.693147
5367,22.746902,10.491593,1.708378,-7.264430,21.137464,-0.916291,16.853589,15.376362,12.887474,41.790121,...,-2.407946,-3.196625,28.298850,-3.912023,5.429346,0.693147,4.912655,1.304569,5.965018,2.564949
5370,24.459998,9.780392,1.928619,-7.130899,21.415475,-1.237874,14.960469,14.803349,10.667699,36.619467,...,-2.465104,-7.013116,22.497059,-0.673345,2.708050,0.693147,2.197225,2.737324,4.537534,1.386294
5376,25.253606,9.979195,1.940179,-6.725434,21.852409,-1.203973,14.789755,14.892019,9.857758,36.515631,...,-1.500584,-8.111728,17.401503,-4.605170,1.791759,1.386294,1.386294,3.165011,4.331391,2.079442


In [94]:
data0 = logtrain_cleaned

X = data0.drop(columns = 'y')
y = data0.y

In [98]:
y

0       1.609438
2       2.397895
4       1.609438
5       0.000000
7       1.098612
          ...   
5365    0.693147
5367    2.564949
5370    1.386294
5376    2.079442
5378    2.564949
Name: y, Length: 2857, dtype: float64

In [103]:
data1 = train_full

X = data1.drop(columns = 'y')
y = data1.y

y

0        5
2       11
4        5
5        1
7        3
        ..
5365     2
5367    13
5370     4
5376     8
5378    13
Name: y, Length: 2857, dtype: int64

In [104]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score

# Assuming your data is stored in X and y

knn = KNeighborsRegressor(n_neighbors=100)  # Initialize the KNN model

# Perform k-fold cross-validation
scores = cross_val_score(knn, X, y, 
                         cv=100, scoring='neg_mean_squared_error')

# Calculate the root mean squared error (RMSE) from the cross-validation scores
rmse_scores = (-scores) ** 0.5

# Print the RMSE for each fold
for fold, rmse in enumerate(rmse_scores):
    print(f"Fold {fold+1} RMSE: {rmse}")

# Calculate the average RMSE across all folds
avg_rmse = rmse_scores.mean()
print(f"Average RMSE: {avg_rmse}")

Fold 1 RMSE: 3.9284931743213773
Fold 2 RMSE: 13.821937086659968
Fold 3 RMSE: 9.913398109072666
Fold 4 RMSE: 15.344442102868252
Fold 5 RMSE: 6.119141533209398
Fold 6 RMSE: 19.945772605431614
Fold 7 RMSE: 14.004793637450936
Fold 8 RMSE: 6.581464830138108
Fold 9 RMSE: 5.974948275638185
Fold 10 RMSE: 4.953826803593359
Fold 11 RMSE: 5.988651912086554
Fold 12 RMSE: 15.421137531278966
Fold 13 RMSE: 8.209756811048164
Fold 14 RMSE: 10.47638065956645
Fold 15 RMSE: 8.171101094922781
Fold 16 RMSE: 9.51398879327807
Fold 17 RMSE: 8.427558447101411
Fold 18 RMSE: 22.255459306999647
Fold 19 RMSE: 5.817333492719602
Fold 20 RMSE: 6.608038469454692
Fold 21 RMSE: 13.440798159797422
Fold 22 RMSE: 18.011716301838952
Fold 23 RMSE: 5.487335261842679
Fold 24 RMSE: 8.838119241339637
Fold 25 RMSE: 9.591428285282863
Fold 26 RMSE: 9.609036336196509
Fold 27 RMSE: 8.600195867857735
Fold 28 RMSE: 5.183175035465797
Fold 29 RMSE: 5.363086797731322
Fold 30 RMSE: 7.208172564814048
Fold 31 RMSE: 14.463961039961262
Fold 32 

In [99]:
#INITIAL TRAIN-TEST SPLIT
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                    random_state=42)
y_test

2972    1.609438
3263    2.197225
1478    2.302585
3238    1.609438
747     1.386294
          ...   
2273    1.386294
4104    1.098612
2870    3.295837
4341    2.639057
40      1.609438
Name: y, Length: 858, dtype: float64

Decided to Move on from KNN, only for the fact that nearest neighbors seems to try to predict based on 

In [100]:
import tensorflow as tf
from tensorflow import keras
tf.config.run_functions_eagerly(True)

# Define the model architecture
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(658,)),  # Adjust the number of neurons and activation function as needed
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)  # Output layer for regression, no activation function
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [61]:
#PCA MODEL

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

X_transformed = StandardScaler().fit_transform(X_train)

pca = PCA(n_components=12)

principalComponents = pca.fit_transform(X_transformed)

principalDf = pd.DataFrame(data = principalComponents
             , columns = logtrain_cleaned.columns)

ValueError: Shape of passed values is (2857, 12), indices imply (2857, 658)

In [ ]:
#correlation_matrix = train_cleaned.corr()['y']
#sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
#plt.show()

### NOTES TAKEN OVER MY KAGGLE

#neighborhoods and accessibility
#what topics? type of program (sports, stem, arts, etc etc.)

#data science job: do a lot of LEETCODE!
#SQL leetcode and python code.
#Expedia and who else

#interview
#KPI's and metrics you can generate?
#conversion, return rate, etc etc.
#can you write a sql query for that?

#clear the behavioral, and statistic interview
#sql inteview test within a time limit.
#lots of counter questions, trying to trick you and help you.
#find if you're a good fit, and best person to have role.

#typical work day
#traffic, baseline, lower standard errors.
#interview prep advice? how to share what you did to get he interviews?
#dashboards, experiments, portfolio of work. dashboarding tools and skills.
#for experiments, it depends on the company, data tracking etc etc.

#ask chatgpt lol. does this job.
#TRY TO GET REFERRALS!!!
#career in masters, dynamic and not bored.
#Let me do SQL! show me I can do SQL!
#progress and let you do expedia.
#know what a type 1 error is.

#lasso and random forest, comparing with correlation.
#PCA Final variable selection, top 10 which has a deep correlation.
#SVM, KNN, boosted tree, elastic net, 
#A lot of non-linearity, investigate that further.
#Stealing ideas from Will Cichowski's group
#better imputation methods. 
#GCP, run your code without your computer
#Tree based is time consuming.